In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import RNNModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from pytorch_lightning.callbacks import Callback, EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna


# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [5]:
# 1. Carregar e preparar os dados
target_columns = [
    'Temperature','Precipitation_log','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

# Carregar os dados
df1 = pd.read_csv("data/cleaned/ground_station_2022-08-03_2022-12-17.csv")
df2 = pd.read_csv("data/cleaned/ground_station_2023-08-02_2024-01-11.csv")
df3 = pd.read_csv("data/cleaned/ground_station_2024-05-29_2024-09-16.csv")

print(len(df1), len(df2), len(df3))

6515 7794 5253


In [6]:
encoders = {
    "cyclic":{"future":["month","dayofyear","day","hour","minute"]},
    "transformer": Scaler()
}

In [7]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

In [17]:

def objective(trial):

    input_chunk_length = trial.suggest_int("input_chunk_length", 12, 64)
    training_length = trial.suggest_int("training_length", input_chunk_length+1, input_chunk_length+64)

    print(f"input_chunk_length: {input_chunk_length}, training_length: {training_length}")

    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
            "accelerator": "auto",
            "callbacks": [early_stopper],
    }
    n = int(len(df1)*0.8)
    train_df_fold, val_df_fold = df1.iloc[:n], df1.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='30min')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='30min')

    print("\nTreinando o modelo...")
    print(f"Train set: {train_fold.shape[0]} samples")
    print(f"Validation set: {val_fold.shape[0]} samples")
    
    scaler = Scaler()
    # Ajustar o scaler apenas nos dados de treino para evitar data leakage
    scaler = scaler.fit(train_fold)
    # Transformar todas as séries
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)
    

    model = RNNModel(
        model="GRU",
        model_name=f"model",
        input_chunk_length=input_chunk_length,
        hidden_dim=128,
        n_rnn_layers=2,
        n_epochs=100,
        batch_size=32,
        dropout=0.4,
        add_encoders=encoders,
        training_length=training_length,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,    
        show_warnings=True,
        force_reset=True,
        optimizer_kwargs={"lr": 0.0003, "weight_decay": 2e-5},  # Definir taxa de aprendizado
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True
    )
    
    model.load_from_checkpoint(f"/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/model/", best=True)

    forecasts = model.historical_forecasts(
        train_scaled,
        forecast_horizon=12,
        stride=1,
        retrain=False,
        verbose=True
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    inutil,s = train_scaled.split_after(0.8)
    smape_val = np.nan
    try:
        smape_val = smape(s,forecasts_t)
    except ValueError as e:
        print(f"Erro ao calcular SMAPE: {e}")
        smape_val = np.nan
    print(f"O SMAPE desse fold foi {smape_val}")

    return smape_val if smape_val != np.nan else float("inf")


In [18]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [19]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20, callbacks=[print_callback])

[I 2025-05-10 22:42:33,533] A new study created in memory with name: no-name-e29302cf-85a2-4cbf-852b-79c6717b0706
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | HuberLoss        | 0      | train
1 | train_criterion | HuberLoss        | 0      | train
2 | val_criterion   | HuberLoss        | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | GRU              | 155 K  | train
6 | V               | Linear           | 1.0 K  | train
-------------------------------------------------------------
156 K     Trainable params
0         Non-trainable params
156 K     Total params
0.628     Total estimated model params size (MB)
7         Modules in trai

input_chunk_length: 47, training_length: 75

Treinando o modelo...
Train set: 5212 samples
Validation set: 1303 samples


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.001 >= min_delta = 0.001. New best score: 0.007

Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2025-05-10 22:57:46,650] Trial 7 failed with parameters: {'input_chunk_length': 63, 'training_length': 114} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._run_stage()
            

NameError: name 'exit' is not defined